## Demo 3: HKR classifier on MNIST dataset
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deel-ai/deel-lip/blob/master/doc/notebooks/demo3.ipynb)

This notebook will demonstrate learning a binary task on the MNIST0-8 dataset.

In [ ]:
# pip install deel-lip -qqq

In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.models import Sequential

from deel.lip.layers import (
    SpectralConv2D,
    SpectralDense,
    FrobeniusDense,
    ScaledL2NormPooling2D,
)
from deel.lip.activations import MaxMin, GroupSort, GroupSort2, FullSort
from deel.lip.losses import HKR, KR, HingeMargin

### data preparation

For this task we will select two classes: 0 and 8. Labels are changed to {-1,1}, wich is compatible
with the Hinge term used in the loss.

In [3]:
from tensorflow.keras.datasets import mnist

# first we select the two classes
selected_classes = [0, 8]  # must be two classes as we perform binary classification


def prepare_data(x, y, class_a=0, class_b=8):
    """
    This function convert the MNIST data to make it suitable for our binary classification
    setup.
    """
    # select items from the two selected classes
    mask = (y == class_a) + (
        y == class_b
    )  # mask to select only items from class_a or class_b
    x = x[mask]
    y = y[mask]
    x = x.astype("float32")
    y = y.astype("float32")
    # convert from range int[0,255] to float32[-1,1]
    x /= 255
    x = x.reshape((-1, 28, 28, 1))
    # change label to binary classification {-1,1}
    y[y == class_a] = 1.0
    y[y == class_b] = -1.0
    return x, y


# now we load the dataset
(x_train, y_train_ord), (x_test, y_test_ord) = mnist.load_data()

# prepare the data
x_train, y_train = prepare_data(
    x_train, y_train_ord, selected_classes[0], selected_classes[1]
)
x_test, y_test = prepare_data(
    x_test, y_test_ord, selected_classes[0], selected_classes[1]
)

# display infos about dataset
print(
    "train set size: %i samples, classes proportions: %.3f percent"
    % (y_train.shape[0], 100 * y_train[y_train == 1].sum() / y_train.shape[0])
)
print(
    "test set size: %i samples, classes proportions: %.3f percent"
    % (y_test.shape[0], 100 * y_test[y_test == 1].sum() / y_test.shape[0])
)


train set size: 11774 samples, classes proportions: 50.306 percent
test set size: 1954 samples, classes proportions: 50.154 percent


### Build lipschitz Model

Let's first explicit the paremeters of this experiment

In [4]:
# training parameters
epochs = 10
batch_size = 128

# network parameters
activation = GroupSort  # ReLU, MaxMin, GroupSort2

# loss parameters
min_margin = 2.0
alpha = 10.0



Now we can build the network.
Here the experiment is done with a MLP. But `Deel-lip` also provide state of the art 1-Lipschitz convolutions.

In [5]:
K.clear_session()
# helper function to build the 1-lipschitz MLP
wass = Sequential(
    layers=[
        Input((28, 28, 1)),
        Flatten(),
        SpectralDense(32, GroupSort2(), use_bias=True),
        SpectralDense(16, GroupSort2(), use_bias=True),
        FrobeniusDense(1, activation=None, use_bias=False),
    ],
    name="lipModel",
)
wass.summary()

2022-07-19 10:22:47.009179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 10:22:47.034880: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 10:22:47.035148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 10:22:47.035589: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Model: "lipModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 spectral_dense (SpectralDen  (None, 32)               50241     
 se)                                                             
                                                                 
 spectral_dense_1 (SpectralD  (None, 16)               1057      
 ense)                                                           
                                                                 
 frobenius_dense (FrobeniusD  (None, 1)                32        
 ense)                                                           
                                                                 
Total params: 51,330
Trainable params: 25,664
Non-trainable params: 25,666
_________________________________________________

In [6]:
optimizer = Adam(lr=0.001)

/data/home/franck.mamalet/.conda/envs/deel-tf2.8/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
# as the output of our classifier is in the real range [-1, 1], binary accuracy must be redefined
def HKR_binary_accuracy(y_true, y_pred):
    S_true = tf.dtypes.cast(tf.greater_equal(y_true[:, 0], 0), dtype=tf.float32)
    S_pred = tf.dtypes.cast(tf.greater_equal(y_pred[:, 0], 0), dtype=tf.float32)
    return binary_accuracy(S_true, S_pred)


In [8]:
wass.compile(
    loss=HKR(
        alpha=alpha, min_margin=min_margin
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        KR(),  # shows the KR term of the loss
        HingeMargin(min_margin=min_margin),  # shows the hinge term of the loss
        HKR_binary_accuracy,  # shows the classification accuracy
    ],
    optimizer=optimizer,
)

### Learn classification on MNIST

Now the model is build, we can learn the task.

In [9]:
wass.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=batch_size,
    shuffle=True,
    epochs=epochs,
    verbose=1,
)

Epoch 1/10
92/92 [==============================] - 2s 10ms/step - loss: -3.2245 - KR: 4.4264 - HingeMargin: 0.1202 - HKR_binary_accuracy: 0.9507 - val_loss: -5.2971 - val_KR: 5.7282 - val_HingeMargin: 0.0449 - val_HKR_binary_accuracy: 0.9825
Epoch 2/10
92/92 [==============================] - 0s 5ms/step - loss: -5.3499 - KR: 5.7587 - HingeMargin: 0.0409 - HKR_binary_accuracy: 0.9857 - val_loss: -5.4849 - val_KR: 5.8323 - val_HingeMargin: 0.0364 - val_HKR_binary_accuracy: 0.9835
Epoch 3/10
92/92 [==============================] - 0s 5ms/step - loss: -5.4647 - KR: 5.8403 - HingeMargin: 0.0376 - HKR_binary_accuracy: 0.9865 - val_loss: -5.4293 - val_KR: 5.9406 - val_HingeMargin: 0.0515 - val_HKR_binary_accuracy: 0.9796
Epoch 4/10
92/92 [==============================] - 0s 5ms/step - loss: -5.5039 - KR: 5.8894 - HingeMargin: 0.0385 - HKR_binary_accuracy: 0.9856 - val_loss: -5.5837 - val_KR: 5.9144 - val_HingeMargin: 0.0352 - val_HKR_binary_accuracy: 0.9850
Epoch 5/10
92/92 [=============

As we can see the model reach a very decent accuracy on this task.